In [3]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Load environment variables
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key
if not api_key:
    print("No API key was found - please check your .env file!")
elif not api_key.startswith("sk-proj-"):
    print("Invalid API key format - please check!")
elif api_key.strip() != api_key:
    print("API key contains extra spaces - please remove them!")
else:
    print("API key loaded successfully!")
    
openai = OpenAI(api_key=api_key)

# Headers for requests
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:120.0) Gecko/20100101 Firefox/120.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",  # Appears as if you're coming from Google
    "Connection": "keep-alive",
    "DNT": "1",  # Do Not Track request
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
}

class Website:

    def __init__(self, url):
        """
        Initialize the website object and fetch content.
        """
        self.url = url
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Raise an error for bad responses
        except requests.RequestException as e:
            print(f'Failed to fetch website: {e}')
            self.title = 'Error fetching website'
            self.text = ''
            return
        
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = 'No readable content found'

url = input('Write down your website URL here: \n')
website = Website(url)

# Step 1: Create prompts

system_prompt = "You are an senior research scientist that analyzes the contents of a website \
    and provides a short summary, ignoring text that might be navigation related. \
    Respond in markdown."
user_prompt = f"You are looking at a website titled {website.title}. \n\
    The contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n\
    {website.text}"

# Step 2: Construct messages list

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
] 

# Step 3: Call OpenAI API
try:
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages
    )
    
# Step 4: print the result
    display(Markdown(response.choices[0].message.content))
except Exception as e:
    print(f'OpenAI API call failed: {e}')

API key loaded successfully!
Write down your website URL here: 
https://edwarddonner.com


# Summary of the "Home - Edward Donner" Website

The website is a personal platform for Edward Donner, who is a co-founder and CTO of Nebula.io. It highlights his interests in coding, experimenting with large language models (LLMs), and his background in DJing and music production. He has a previous venture, an AI startup named untapt, which was acquired in 2021.

## Recent Announcements
- **January 23, 2025**: LLM Workshop – Hands-on with Agents – resources.
- **December 21, 2024**: Welcome, SuperDataScientists!
- **November 13, 2024**: Mastering AI and LLM Engineering – Resources.
- **October 16, 2024**: From Software Engineer to AI Data Scientist – resources.

The site emphasizes the innovative work done with proprietary LLMs in the talent discovery field and aims to foster a community of like-minded individuals.